In [1]:
import pandas as pd

In [2]:
borrower_data = pd.read_csv(r'C:\Users\Windows\Downloads\DataScience-Assignment\DataScience-Assignment\Borrower_Payment_History.csv')

In [3]:
borrower_data

,CustomerID,LoanID,DateReported,YearMonthReported,InstallmentAmount,TotalOutstandingBalance,OverdueAmount,DPDStatus,LoanStatus,SanctionAmount,InterestRate,LoanTerm,LoanType,Income,EmploymentStatus,No_of_days_past_due
0,1,d5cbc0b8-b8a9-43b0-b482-abec237835b6,22:27.1,2021-06,13411.52,482814.85,0.00,1,OPEN,438126,10.20,36,Housing Finance,51967,Employed,0
1,1,d5cbc0b8-b8a9-43b0-b482-abec237835b6,22:27.1,2021-07,13411.52,469403.33,0.00,1,OPEN,438126,10.20,36,Housing Finance,51967,Employed,0
2,1,d5cbc0b8-b8a9-43b0-b482-abec237835b6,22:27.1,2021-08,13411.52,455991.80,0.00,1,OPEN,438126,10.20,36,Housing Finance,51967,Employed,0
3,1,d5cbc0b8-b8a9-43b0-b482-abec237835b6,22:27.1,2021-09,13411.52,442580.28,0.00,1,OPEN,438126,10.20,36,Housing Finance,51967,Employed,0
4,1,d5cbc0b8-b8a9-43b0-b482-abec237835b6,22:27.1,2021-10,13411.52,429168.76,0.00,1,OPEN,438126,10.20,36,Housing Finance,51967,Employed,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36995,1000,801fda65-b954-4fd2-830e-2a66f3066d29,22:33.0,2024-02,6576.96,46038.75,26307.85,5,OPEN,215540,9.85,36,Housing Finance,44393,Employed,95
36996,1000,801fda65-b954-4fd2-830e-2a66f3066d29,22:33.0,2024-03,6576.96,39461.78,26307.85,5,OPEN,215540,9.85,36,Housing Finance,44393,Employed,95
36997,1000,801fda65-b954-4fd2-830e-2a66f3066d29,22:33.0,2024-04,6576.96,32884.82,26307.85,5,OPEN,215540,9.85,36,Housing Finance,44393,Employed,95
36998,1000,801fda65-b954-4fd2-830e-2a66f3066d29,22:33.0,2024-05,6576.96,0.00,0.00,1,CLOSE,215540,9.85,36,Housing Finance,44393,Employed,0


In [4]:
# Data Preprocessing: Creating a target variable 'default_in_next_12_months'

# Create a target column to indicate if the borrower has ever been 90+ days overdue in the next 12 months
borrower_data['default_in_next_12_months'] = borrower_data['No_of_days_past_due'].apply(lambda x: 1 if x >= 90 else 0)

# Step 2: Basic statistics to verify the distribution of the target variable
default_stats = borrower_data['default_in_next_12_months'].value_counts()

default_stats

default_in_next_12_months
0    34705
1     2295
Name: count, dtype: int64

In [5]:
borrower_data[borrower_data['CustomerID']==1].default_in_next_12_months.value_counts()

default_in_next_12_months
0    33
1     4
Name: count, dtype: int64

In [6]:
borrower_data['prev_late_payments'] = borrower_data.groupby('CustomerID')['default_in_next_12_months'].transform(lambda x:x.sum())

In [8]:
borrower_data['loan_to_income'] = borrower_data['TotalOutstandingBalance'] / borrower_data['Income']

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
X = borrower_data[['InstallmentAmount', 'TotalOutstandingBalance', 'prev_late_payments', 'loan_to_income','InterestRate']]
y = borrower_data['default_in_next_12_months']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
model_details = {
    'svm': {
        'model': SVC(gamma='auto'),
        'params' : {
            'C': [1], #[1,10,20],
            'kernel': ['rbf'], #'linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5]
        }
    }
}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Initialize an empty dictionary to store the best models
best_models = {}

# Iterate over each model and perform GridSearchCV
for model_name, config in model_details.items():
    model = config['model']
    params = config['params']
    
    # Perform grid search
    grid_search = GridSearchCV(model, params, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)
    
    # Save the best model
    best_models[model_name] = grid_search.best_estimator_

    # Print the best parameters and score for each model
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best score for {model_name}: {grid_search.best_score_}")
